In [9]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4", api_key=api_key)

# Method 1: Single comprehensive prompt (most efficient)
comprehensive_prompt = PromptTemplate(
    input_variables=["content", "max_words"],
    template="""
    Please perform the following tasks in sequence:
    1. Summarize the following content in {max_words} words or fewer, focusing on main points
    2. Translate the summary to Swahili
    3. Format the translation in uppercase
    
    Content: {content}
    
    Provide only the final uppercase Swahili text:
    """
)

# Method 2: LCEL chain (modern LangChain approach)
summary_prompt = PromptTemplate(
    input_variables=["content", "max_words"],
    template="""Summarize the following content in {max_words} words or fewer:
    {content}
    Summary:"""
)

translate_prompt = PromptTemplate(
    input_variables=["summary"],
    template="""Translate to Swahili: {summary}
    Translation:"""
)

format_prompt = PromptTemplate(
    input_variables=["translation"],
    template="""Make this uppercase: {translation}
    Result:"""
)

# LCEL chain
chain = (
    summary_prompt 
    | llm 
    | (lambda x: {"summary": x})
    | translate_prompt 
    | llm 
    | (lambda x: {"translation": x})
    | format_prompt 
    | llm
)

# Test content
content = """
Once upon a time in a small village, a curious boy named Kito dreamed of exploring the world beyond the hills. Every day, he listened to stories from travelers who passed through, imagining faraway lands filled with wonders. One morning, Kito packed a small bag and set off on an adventure. Along the way, he met new friends, overcame challenges, and discovered the beauty of nature. After many days, he returned home with stories of his own, inspiring the village children to dream big. Kito learned that courage and curiosity could open doors to the greatest journeys.
"""

# Method 1 - Single call (recommended)
print("Method 1 - Single comprehensive prompt:")
result1 = comprehensive_prompt.format(content=content, max_words="50")
final_result1 = llm.invoke(result1)
print(final_result1.content)

print("\n" + "="*50 + "\n")

# Method 2 - LCEL chain
print("Method 2 - LCEL chain:")
result2 = chain.invoke({"content": content, "max_words": "50"})
print(result2.content)

Method 1 - Single comprehensive prompt:
WAKATI MMOJA KATIKA KIJIJI KIDOGO, MVULANA MGENI ALIYEITWA KITO ALIOTA NDOTO YA KUTEMBELEA ULIMWENGU NYUMA YA MILIMA. KILA SIKU, ALISIKILIZA HADITHI ZA WASAFIRI WALIOPITA, AKIWAZIA NCHI ZA MBALI ZILIZOJAA MAJABU. SIKU MOJA ASUBUHI, KITO ALIFUNGASHA MFUKO MDOGO NA KUANZA SAFARI YA UGUNDUZI. KATIKA SAFARI YAKE, ALIKUTANA NA MARAFIKI WAPYA, AKASHINDA CHANGAMOTO, NA KUGUNDUA UZURI WA ASILI. BAADA YA SIKU NYINGI, ALIRUDI NYUMBANI NA HADITHI ZAKE, AKIHAMASISHA WATOTO WA KIJIJI KUOTA NDOTO KUBWA. KITO ALIGUNDUA KWAMBA UJASIRI NA UDHADISI VINAFUNGUA MILANGO KWA SAFARI KUBWA ZAIDI.



Method 2 - LCEL chain:
CONTENT='KIJANA ALIYEITWA KITO KUTOKA KIJIJINI ALIKUWA NA NDOTO YA KUSAFIRI MBALI NA NYUMBANI KWAKE. AKIHAMASISHWA NA HADITHI KUTOKA KWA WASAFIRI, ALIANZA SAFARI YA HATARI, AKIFANYA MARAFIKI WAPYA NA KUPITIA UZURI WA ASILI. ALIPORUDI, ALISHIRIKI UZOEFU WAKE, KUWATIA MOYO WATOTO WENGINE KUFIKIA MALANGO MAKUBWA. ALIGUNDUA KUWA UJASIRI NA UDADISI HUSABABI